In [10]:
import numpy as np
import matplotlib as plt
import torch
import gpytorch

## Constant for Cost function
THRESHOLD = 0.5
W1 = 1
W2 = 20
W3 = 100
W4 = 0.04


def cost_function(true, predicted):
    """
        true: true values in 1D numpy array
        predicted: predicted values in 1D numpy array

        return: float
    """
    cost = (true - predicted)**2

    # true above threshold (case 1)
    mask = true > THRESHOLD
    mask_w1 = np.logical_and(predicted>=true,mask)
    mask_w2 = np.logical_and(np.logical_and(predicted<true,predicted >=THRESHOLD),mask)
    mask_w3 = np.logical_and(predicted<THRESHOLD,mask)

    cost[mask_w1] = cost[mask_w1]*W1
    cost[mask_w2] = cost[mask_w2]*W2
    cost[mask_w3] = cost[mask_w3]*W3

    # true value below threshold (case 2)
    mask = true <= THRESHOLD
    mask_w1 = np.logical_and(predicted>true,mask)
    mask_w2 = np.logical_and(predicted<=true,mask)

    cost[mask_w1] = cost[mask_w1]*W1
    cost[mask_w2] = cost[mask_w2]*W2

    reward = W4*np.logical_and(predicted < THRESHOLD,true<THRESHOLD)
    if reward is None:
        reward = 0
    return np.mean(cost) - np.mean(reward)

"""
Fill in the methods of the Model. Please do not change the given methods for the checker script to work.
You can add new methods, and make changes. The checker script performs:


    M = Model()
    M.fit_model(train_x,train_y)
    prediction = M.predict(test_x)

It uses predictions to compare to the ground truth using the cost_function above.
"""



# The code is copyed from the first tutorial of Gpytorch
# link: https://docs.gpytorch.ai/en/v1.1.1/examples/01_Exact_GPs/Simple_GP_Regression.html

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)





class Model():

    def __init__(self):
        
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()
       

    def predict(self, test_x):
        test_x = torch.Tensor(test_x)
        self.model.eval()
        self.likelihood.eval()

        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            observed_pred = self.likelihood(self.model(test_x))
        
        return observed_pred.mean.numpy()

    def fit_model(self, train_x, train_y):
        
        training_iter = 10
        
        train_x = torch.Tensor(train_x)
        train_y = torch.Tensor(train_y)
        
        self.model = ExactGPModel(train_x, train_y, self.likelihood)
        
        self.model.train()
        self.likelihood.train()

        # Use the adam optimizer
        optimizer = torch.optim.Adam([
        {'params': self.model.parameters()},  # Includes GaussianLikelihood parameters
        ], lr=0.1)

        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)
        
        for i in range(training_iter):
            # Zero gradients from previous iteration
            optimizer.zero_grad()
            # Output from model
            output = self.model(train_x)
            # Calc loss and backprop gradients
            loss = -mll(output, train_y)
            loss.backward()
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                self.model.covar_module.base_kernel.lengthscale.item(),
                self.model.likelihood.noise.item()
            ))
            optimizer.step()


In [ ]:

train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

M = Model()
M.fit_model(train_x, train_y)
prediction = M.predict(test_x)

print(prediction)

In [ ]:
!conda list

In [ ]:
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')



# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(train_x[:,0], train_x[:,1], s=1, c=train_y)
plt.show()
import matplotlib.pyplot as plt
plt.scatter(test_x[:,0], test_x[:,1])
plt.show()

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import Ridge

import time

In [ ]:
nystroem=Nystroem(gamma=1)
x_transformed = nystroem.fit_transform(train_x[:,1].reshape(-1, 1))

rlf = GaussianProcessRegressor()
rlf.fit(x_transformed, train_y)

In [ ]:
xy = np.mgrid[-0.5:1:0.025, -1:1:0.05].reshape(2,-1).T
xy_trans = nystroem.transform(xy[:,1].reshape(-1, 1))
y_linespace = rlf.predict(xy_trans)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(xy[:,0], xy[:,1], s=10, c=y_linespace)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(xy[:,0], xy[:,1], s=10, c=y_linespace)
plt.show()

In [ ]:
x, std = rlf.predict(transformed_x, return_std=True)
x, std

In [ ]:
pred, std_dev = rlf.predict(val_x, return_std=True)

In [ ]:
from scipy.stats import norm


In [ ]:
prop_pred = np.ones(val_y.shape)
prop_pred[(pred <0.5) & (std_dev > 0)] = norm(pred[(pred <0.5) & (std_dev > 0)], std_dev[(pred <0.5) & (std_dev > 0)]).cdf(0.5)
prop_pred[prop_pred < 0.99]

In [ ]:
pred[prop_pred<0.99] = 0.5
val_x[(pred < 0.5) & (val_y > 0.5)]

In [37]:
cost_function(val_y, pred)

NameError: name 'pred' is not defined

In [ ]:
val_x_trans = nystroem.transform(val_x)
cost_function(val_y, rlf.predict(val_x_trans))

In [2]:
import math
import numpy as np
import torch
import gpytorch
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2


# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')
index = train_x[:, 0]>-0.4
X_train, val_x, y_train, val_y = train_test_split(
    train_x[index], 
    train_y[index], 
    test_size=0.2,
    random_state=42)
train_x = np.concatenate((X_train, train_x[~index]))
train_y = np.append(y_train, train_y[~index])

In [4]:
train_x = torch.Tensor(train_x)
train_y = torch.Tensor(train_y)
val_x = torch.Tensor(val_x)
val_y = torch.Tensor(val_y)

In [5]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [6]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

training_iter = 50

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()


Iter 1/50 - Loss: 0.741   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.705   lengthscale: 0.744   noise: 0.644
Iter 3/50 - Loss: 0.667   lengthscale: 0.798   noise: 0.598
Iter 4/50 - Loss: 0.630   lengthscale: 0.853   noise: 0.554
Iter 5/50 - Loss: 0.591   lengthscale: 0.908   noise: 0.513
Iter 6/50 - Loss: 0.552   lengthscale: 0.959   noise: 0.474
Iter 7/50 - Loss: 0.512   lengthscale: 0.998   noise: 0.437
Iter 8/50 - Loss: 0.471   lengthscale: 1.019   noise: 0.403
Iter 9/50 - Loss: 0.430   lengthscale: 1.024   noise: 0.370
Iter 10/50 - Loss: 0.388   lengthscale: 1.016   noise: 0.340
Iter 11/50 - Loss: 0.345   lengthscale: 1.001   noise: 0.312
Iter 12/50 - Loss: 0.302   lengthscale: 0.982   noise: 0.286
Iter 13/50 - Loss: 0.258   lengthscale: 0.957   noise: 0.261
Iter 14/50 - Loss: 0.214   lengthscale: 0.929   noise: 0.239
Iter 15/50 - Loss: 0.169   lengthscale: 0.901   noise: 0.218
Iter 16/50 - Loss: 0.124   lengthscale: 0.872   noise: 0.199
Iter 17/50 - Loss: 0.078   length

In [61]:
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')
test_x = torch.Tensor(test_x)

In [62]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))



In [63]:

predicted = observed_pred.mean.numpy()

In [47]:
cost_function(true, predicted)


-0.00101813834802858

In [64]:
from scipy.stats import norm
var = observed_pred.variance.detach()
pred_norm = norm(predicted, var.numpy())

In [74]:
pred_new = predicted.copy()
prop = 1-pred_norm.cdf([0.5]*len(predicted))
pred_new[(predicted<0.5) & (prop > 0.3)] = 0.5
print(predicted[(predicted<0.5)& (prop > 0.3)])

[0.35948405 0.41861215 0.3176902  0.33265367 0.37785214 0.33506316
 0.3619866  0.33708006 0.45552596 0.43954518 0.32057878 0.3208769
 0.44594634 0.32575798 0.34865913 0.43650407 0.44257608 0.4456481
 0.4095768  0.45005193 0.31999668 0.3359889  0.39083293 0.33462504
 0.33674854 0.3663136  0.40022853 0.3468207  0.37068143 0.33303675
 0.4559605  0.32236627 0.3880891  0.39630863 0.3701878  0.35681632
 0.34067288 0.42975214 0.37976256 0.34389517 0.3332869  0.34545606
 0.350161   0.3218861  0.40188524 0.35644448 0.3931515  0.3357171
 0.32446507 0.40370932 0.33630204 0.39284942 0.34827277 0.3367366
 0.38755718 0.4638171  0.41204992]


(-0.00101813834802858, -0.00101813834802858)

In [44]:
plt.scatter(test_x[:,0], test_x[:,1], s=50, c=observed_pred.mean)
plt.show()




AttributeError: module 'matplotlib' has no attribute 'scatter'

In [ ]:

plt.scatter(test_x[:,0], test_x[:,1], s=50, c=observed_pred.variance.detach().numpy())
plt.show()



In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])

